In [1]:
from awpy import Demo

import torch
from torch_geometric_temporal.signal import DynamicHeteroGraphTemporalSignal

import pandas as pd
import polars as pl
import numpy as np

import seaborn as sns
import matplotlib.pyplot as plt

from sklearn.preprocessing import MinMaxScaler

import time
import sys
import os

pd.set_option('display.max_columns', 100)
pl.Config.set_tbl_rows(10)

sys.path.append(os.path.abspath('../../package'))

from CS2.graph import TabularGraphSnapshot, HeteroGraphSnapshot, TemporalHeteroGraphSnapshot
from CS2.token import Tokenizer
from CS2.preprocess import Dictionary, NormalizePosition, NormalizeTabularGraphSnapshot, ImputeTabularGraphSnapshot
from CS2.visualize import HeteroGraphVisualizer

### 1. Tabular dataset creation

#### 2024 Matches

In [2]:
# --------------------------------------------------
# PREPARATION
# --------------------------------------------------

MATCH_FOLDER_PATH = '../../data/matches-raw/cs2/2024/'
SAVE_PATH = '../../data/matches_processed/cs2/tabular/2024/'

# Get inferno matches
match_list = os.listdir(MATCH_FOLDER_PATH)
inferno_match_list = [file for file in match_list if 'inferno' in file and 
    '-p1' not in file and 
    '-p2' not in file and 
    '-p3' not in file and 
    '-p4' not in file and 
    '-p5' not in file
]

# Save complete match list and process time
parsed_matches_list = []
process_time_list = []

# Match index
match_index = 0

# Map nodes dataset
nodes = pd.read_csv('../../data/map_graph_model/de_inferno/nodes.csv')



# --------------------------------------------------
# PARSE MATCHES
# --------------------------------------------------

# Parse Inferno matches
for match in inferno_match_list:

    # Start timer
    start_time = time.time()

    # Match path
    match_path = MATCH_FOLDER_PATH + match

    # Create tabular snapshot object
    tg = TabularGraphSnapshot()

    # Create tabular snapshot database
    df, df_dict, active_infernos, active_smokes, active_he_smokes = tg.process_match(
        match_path=match_path,
        player_stats_data_path='../../data/player-stats/scraped-in-2024/2023/norm_player_stats_2023.csv',
        missing_player_stats_data_path='../../data/player-stats/missing_players_df_2024.csv',
        weapon_data_path='../../data/weapon_info/ammo_info.csv',

        ticks_per_second=4,
        numerical_match_id=100000 + match_index,
        num_permutations_per_round=1,
        build_dictionary=True,

        package='polars'
    )

    # Impute missing values
    its = ImputeTabularGraphSnapshot()
    df = its.impute(df)

    # Tokenize match
    tokenizer = Tokenizer()
    df = tokenizer.tokenize_match(df.to_pandas(), 'de_inferno', nodes)

    # Save dataframes
    df.to_csv(SAVE_PATH + match + '_df.csv', index=False)
    df_dict.to_csv(SAVE_PATH + match + '_df_dict.csv', index=False)
    active_infernos.to_csv(SAVE_PATH + match + '_active_infernos.csv', index=False)
    active_smokes.to_csv(SAVE_PATH + match + '_active_smokes.csv', index=False)
    active_he_smokes.to_csv(SAVE_PATH + match + '_active_he_smokes.csv', index=False)

    # Post-process
    match_index += 1
    parsed_matches_list.append(match)



    # --------------------------------------------------
    # LOG AND SAVE TIME
    # --------------------------------------------------
    
    # Log and save time
    end_time = time.time()
    process_time = end_time - start_time
    process_time_list.append(process_time)
    print(match + ' completed in ' + str(process_time) + ' seconds')

AttributeError: 'DataFrame' object has no attribute 'with_row_at_idx'